# YouTube video to audio

- Author: [Pierre Guillou](https://www.linkedin.com/in/pierreguillou/)
- Blog post: [Video-to-Audio | A free Web APP to get mp3 audio file from any YouTube video]()
- Date: 06/12/2023

In [1]:
%%capture
!python3 -m pip install -U yt-dlp
!pip install unidecode
!pip install gradio
!pip install pydub

In [ ]:
import gradio as gr
import re, unidecode
from unidecode import unidecode
import yt_dlp
import os
import pydub
import numpy as np

# no space, punctuation, accent in lower string
def cleanString(string):
    cleanString = unidecode(string)
    # cleanString = re.sub('\W+','_', cleanString)
    cleanString = re.sub(r'[^\w\s]','',cleanString)
    cleanString = cleanString.replace(" ", "_")
    return cleanString.lower()

# from YouTube url to audio file path and sample rate + numpy array
def download_audio(url):

    path_to_folder_audio_mp3 = "./"
    ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': f'{path_to_folder_audio_mp3}%(title)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
        }]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        video_title = info_dict['title']

        # Rename the audio file
        local_link = video_title + ".mp3"
        new_local_link = cleanString(video_title) + ".mp3"
        for filename in os.listdir(path_to_folder_audio_mp3):
            if cleanString(local_link) == cleanString(filename):
                os.rename(os.path.join(path_to_folder_audio_mp3, filename),os.path.join(path_to_folder_audio_mp3, new_local_link))

        # get audio file path
        file_path = path_to_folder_audio_mp3 + new_local_link

    return file_path, file_path

with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Free YouTube URL Video-to-Audio</center></h1>")
    gr.Markdown("<center>Enter the link of any YouTube video to generate its mp3 audio file.</center>")

    input_text_url = gr.Textbox(placeholder='Youtube video URL', label='YouTube URL')
    result_button_audio = gr.Button('Get Audio File')
    output_audio_file = gr.File(label='mp3 audio file')
    output_audio_play = gr.Audio(type="filepath", label="Listen to audio")

    result_button_audio.click(download_audio, inputs = input_text_url, outputs = [output_audio_file, output_audio_play])

demo.queue().launch(debug = False)

# END